In [121]:
import re
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import itertools
from math import inf

# My Solution to Exercise 1

In [85]:
def obtain_graph(source = 'graph.txt', dtype = 'dict'):
    with open(source, 'r') as f:
        data = f.read().splitlines()
    
    nodes = {}
    G_nodes = {}
    for idx, line in enumerate(data):
        node = re.search(r'node%s' % int(idx), line).group()
        vals = re.findall(r'node\d+\s\d+\.{,1}\d+', line)
        nodes[node] = [(entry.split(' ')[0], float(entry.split(' ')[1])) for entry in vals]
        G_nodes[node] = {n: {'weight': w} for n,w in nodes[node]}
    
    if dtype == 'dict':
        return nodes
    elif dtype == 'DiGraph':
        return nx.DiGraph(G_nodes)
    else:
        raise ValueError('Return data type unsupported')
        
def draw_graph(G, lbl=True, arr=False):
    nx.draw(G, with_labels=lbl, arrows=arr)
    plt.show()

In [120]:
txt = 'graph.txt'
graph_dict = obtain_graph(txt, 'dict')
G = obtain_graph(txt, 'DiGraph')
paths = nx.all_simple_paths(G, 'node0', 'node99')

In [101]:
target = nx.bellman_ford(G, 'node0')[1]['node99']

In [167]:
paths = {}
distances = {}
nodes = G.nodes()
edges = nx.to_dict_of_dicts(G)
max_links = len(nodes) - 1
s = 'node0'

for node in nodes:
    distances[node] = 10000
        
distances[s] = 0

for unode in nodes:
    for vnode, weight in edges[unode].items():
        temp_dist = weight['weight'] + distances[unode]
        if temp_dist < distances[vnode]:
            distances[vnode] = temp_dist
            paths[vnode] = unode

In [168]:
distances['node99']

160.55000000000007

In [178]:
current = 'node99'
path = [current]

while current != 'node0':
    current = paths[current]
    path.append(current)

In [180]:
path = reversed(path)

In [182]:
list(path)

['node0',
 'node8',
 'node11',
 'node18',
 'node23',
 'node33',
 'node41',
 'node53',
 'node56',
 'node57',
 'node60',
 'node67',
 'node70',
 'node73',
 'node76',
 'node85',
 'node87',
 'node88',
 'node93',
 'node94',
 'node96',
 'node97',
 'node98',
 'node99']